# Geopandas와 지오코딩

---

&emsp;Geopandas는 [geopy](https://geopy.readthedocs.io/en/stable/) 라이브러리를 통해 지오코딩(geocoding)을 지원합니다. Geopandas의 [`geopandas.tools.geocode()`](https://geopandas.org/en/stable/docs/user_guide/geocoding.html) 함수를 사용하기 위해서는 geopy 라이브러리가 설치되어야 합니다. `geocode()`는 주소(문자열; `string`)로 구성된 리스트(`list`) 또는 `pandas.Series`를 입력값으로 받아 주소와 점 도형(point geometry)으로 구성된 `GeoDataFrame`을 반환합니다.  

&emsp;지오코딩할 주소를 세미콜론(`;`)으로 구분된 텍스트 파일로 만들어 `addresses.txt`로 저장합니다. `addresses.txt` 파일은 아래와 같이 구성되어 있습니다.  

지점;지점주소  
400;서울특별시 강남구일원동 580탄천 물재생센터  
401;서울특별시 서초구서초동 1416번지서초 IC  
402;서울특별시 강동구고덕로 183서울종합직업전문학교  
403;서울특별시 송파구올림픽로 240롯데월드  
. . .

&emsp;Pandas 라이브러리를 이용해 `addresses.txt` 데이터를 읽어오겠습니다. 아래에서 보시다시피, 각 행에 대한 `지점` 열과 주소가 입력된 `지점주소` 열이 있습니다.

In [27]:
import pathlib
import pandas as pd

NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_DIRECTORY = NOTEBOOK_PATH / "data"

addresses = pd.read_csv(DATA_DIRECTORY / "addresses.txt", sep=";")
addresses

,지점,지점주소
0,400,서울특별시 강남구 일원동 580
1,401,서울특별시 서초구 서초동 1416번지
2,402,서울특별시 강동구 고덕로 183
3,403,서울특별시 송파구 올림픽로 240
4,404,서울특별시 강서구 양천로 201
5,405,서울특별시 양천구 목동동로 298
6,406,서울특별시 도봉구 시루봉로 173
7,407,서울특별시 노원구 공릉동 사서함 230-3호 사서함 77호
8,408,서울특별시 동대문구 서울시립대로 163
9,409,서울특별시 중랑구 면목로57길 32


## Photon을 사용한 지오코딩

&emsp;*[Photon](https://photon.komoot.io/)* 지오코더(geocoder)를 사용해 위의 `지점주소`들을 지오코딩 하겠습니다. Photon은 OpenStreetMap 데이터를 사용하는 지오코더 서비스 입니다. Geopandas의 `geocode()` 함수는 기본적으로 photon에 기반한 지오코딩을 지원합니다.  

```{note}
&emsp;[Nominatim 이용 약관](https://operations.osmfoundation.org/policies/nominatim/)에 따라, 사용자는 1초당 1회보다 많이 API 요청을 보낼 수 없습니다. 그리고 API 요청 시 **`user-agent`**에 대한 정보를 필수적으로 입력해야 합니다.  

&emsp;주소를 조회하는 작업은 꽤 많은 연산을 필요로 하는 데이터베이스 작업입니다. 이러한 이유 때문에 공용 및 무료 Nominatim API를 통해 응답을 얻기까지 때때로 오랜 시간이 소요될 수 있습니다. 따라서, 여기에서는 파라미터 `timeout=10`을 추가하여 API 응답을 최대 10초 동안 기다리도록 설정합니다.
```

In [28]:
import geopandas as gpd

geocoded_addresses = gpd.tools.geocode(addresses["지점주소"], # 지오코딩할 주소
                                       provider="photon", # 지오코더
                                       user_agent="test", # user-agent 설정 필수 (Nominatim)
                                       timeout=10
                                      )
geocoded_addresses

,geometry,address
0,POINT (127.08775 37.47971),"서울특별시아동학대예방센터, 광평로34길, 06352, 광평로34길, 서울, 대한민국"
1,POINT (127.02191 37.47954),"서울특별시데이터센터, 남부순환로340길, 06724, 남부순환로340길, 서울, 대한민국"
2,POINT (127.14557 37.55604),"서울특별시 동부기술교육원, 183, 고덕로, 05235, 고덕로, 고덕동, 대한민국"
3,POINT (127.09824 37.51150),"롯데월드, 240, 올림픽로, 05554, 올림픽로, 잠실동, 대한민국"
4,POINT (126.81027 37.57468),"월드메르디앙 201동, 양천로1길, 07602, 양천로1길, 서울, 대한민국"
5,POINT (126.87355 37.52290),"405-298, 목동동로12길, 08005, 목동동로12길, 서울, 대한민국"
6,POINT (127.02772 37.65291),"서울특별시립도봉도서관, 시루봉로, 01375, 시루봉로, 쌍문4동, 대한민국"
7,POINT EMPTY,None
8,POINT (127.05918 37.58302),"서울시립대학교, 163, 서울시립대로, 02504, 서울시립대로, 서울특별시, 대한민국"
9,POINT (127.08852 37.59191),"32, 면목로84길, 02162, 면목로84길, 서울시 중랑구, 대한민국"


&emsp;그 결과, `지점주소`를 지오코딩하여 얻은 좌표 `geometry`열과 `address` 열이 포함된 `GeoDataFrame`을 얻었습니다. 이 중 `geometry`열은 지리공간 데이터 형식인 `shapely.geometry.Point`로 구성됩니다.  

&emsp;기존의 입력 데이터셋인 `addresses`와 결합하기 위해 pandas의 [조인(join)](https://pandas.pydata.org/docs/user_guide/merging.html) 기능을 사용할 수 있습니다.

## 데이터프레임 조인 (DataFrame Join)

&emsp;두 개 이상의 데이터프레임 또는 테이블의 데이터를 조인하는 작업은 (공간)데이터 분석 워크플로(workflow)에서 대다수의 경우 수행됩니다. 이전 단원의 내용을 떠올려 보겠습니다. 테이블들을 결합하기 위해 공통되는 <b>키(key)</b> 속성이 있는 경우, `merge()` 함수를 사용하여 간단하게 수행할 수 있습니다.  

&emsp;그러나 때로는 <b>인덱스(index)</b>에 기반하여 두 데이터프레임을 조인하는 것이 유용합니다. 이 경우, 두 데이터프레임은 **동일한 수의 행**을 가져야 하며, **동일한 인덱스를 공유**해야 합니다.  

&emsp;여기에서는 원본 데이터프레임인 `addresses`와 지오코딩 결과인 `geocoded_addresses`의 레코드 순서가 동일하므로, 인덱스에 기반한 조인 방식을 사용하여 두 개의 데이터프레임을 행별로 조인합니다. 참고로 `join()` 함수는 아무런 설정을 하지 않을 경우, 인덱스에 기반하여 두 개의 데이터프레임을 조인합니다. 

&emsp;새로운 데이터프레임인 `geocoded_addresses_with_id`은 원본 데이터프레임에 있었던 모든 열과 지오코딩 결과인 `geometry` 열, `address` 열도 포함되어 있습니다.  

In [29]:
geocoded_addresses_with_id = geocoded_addresses.join(addresses)
geocoded_addresses_with_id

,geometry,address,지점,지점주소
0,POINT (127.08775 37.47971),"서울특별시아동학대예방센터, 광평로34길, 06352, 광평로34길, 서울, 대한민국",400,서울특별시 강남구 일원동 580
1,POINT (127.02191 37.47954),"서울특별시데이터센터, 남부순환로340길, 06724, 남부순환로340길, 서울, 대한민국",401,서울특별시 서초구 서초동 1416번지
2,POINT (127.14557 37.55604),"서울특별시 동부기술교육원, 183, 고덕로, 05235, 고덕로, 고덕동, 대한민국",402,서울특별시 강동구 고덕로 183
3,POINT (127.09824 37.51150),"롯데월드, 240, 올림픽로, 05554, 올림픽로, 잠실동, 대한민국",403,서울특별시 송파구 올림픽로 240
4,POINT (126.81027 37.57468),"월드메르디앙 201동, 양천로1길, 07602, 양천로1길, 서울, 대한민국",404,서울특별시 강서구 양천로 201
5,POINT (126.87355 37.52290),"405-298, 목동동로12길, 08005, 목동동로12길, 서울, 대한민국",405,서울특별시 양천구 목동동로 298
6,POINT (127.02772 37.65291),"서울특별시립도봉도서관, 시루봉로, 01375, 시루봉로, 쌍문4동, 대한민국",406,서울특별시 도봉구 시루봉로 173
7,POINT EMPTY,None,407,서울특별시 노원구 공릉동 사서함 230-3호 사서함 77호
8,POINT (127.05918 37.58302),"서울시립대학교, 163, 서울시립대로, 02504, 서울시립대로, 서울특별시, 대한민국",408,서울특별시 동대문구 서울시립대로 163
9,POINT (127.08852 37.59191),"32, 면목로84길, 02162, 면목로84길, 서울시 중랑구, 대한민국",409,서울특별시 중랑구 면목로57길 32


&emsp;`join()` 함수 출력(output)의 데이터 타입은 `geopandas.GeoDataFrame` 입니다.

In [30]:
type(geocoded_addresses_with_id)

geopandas.geodataframe.GeoDataFrame

```{note}
&emsp;`geocoded_addresses.join(addresses)`와 반대로 `address.join(geocoded_addresses)`처럼 조인하는 경우, 출력 데이터 타입은 `geopandas.GeoDataFrame`이 아닌 `pandas.DataFrame`이 됩니다.
```

&emsp;마지막으로 새 데이터프레임을 *GeoPackage* 형식(format)의 지리공간 파일로 간단하게 저장할 수 있습니다.

In [31]:
geocoded_addresses_with_id.to_file(DATA_DIRECTORY / "addresses.gpkg")

---

```{raw} html
<script src="https://utteranc.es/client.js"
        repo="Kwan-Gu/geospatial_analysis"
        issue-term="pathname"
        theme="preferred-color-scheme"
        crossorigin="anonymous"
        async>
</script>
```